## Prompt Formatting Sample

This notbook is intended to showcase how to correctly format a Llama 3 prompt.

Open in <a href="https://colab.research.google.com/github/varunfb/llama-recipes/blob/main/tools/prompt_generator/prompt_sample.ipynb"><img data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" src="https://camo.githubusercontent.com/f5e0d0538a9c2972b5d413e0ace04cecd8efd828d133133933dfffec282a4e1b/68747470733a2f2f636f6c61622e72657365617263682e676f6f676c652e636f6d2f6173736574732f636f6c61622d62616467652e737667"></a>

In [ ]:
# !pip install gradio

In [4]:
from typing import (
    List,
    Literal,
    Sequence,
    TypedDict,
)

Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str


Dialog = Sequence[Message]

class ChatFormat:

    @staticmethod
    def encode_header( message: Message) -> List[str]:
        prompts = []
        prompts.append("<|start_header_id|>")
        prompts.extend(message["role"])
        prompts.append("<|end_header_id|>")
        prompts.extend("\n\n")
        return prompts

    @staticmethod
    def encode_message(message: Message) -> List[str]:
        prompts = ChatFormat.encode_header(message)
        prompts.extend(message["content"].strip())
        prompts.append("<|eot_id|>")
        return prompts

    @staticmethod
    def encode_dialog_prompt(dialog: Dialog) -> str:
        prompts = ["<|begin_of_text|>"]
        for message in dialog:
            prompts.extend(ChatFormat.encode_message(message))
        # Add the start of an assistant message for the model to complete.
        prompts.extend(ChatFormat.encode_header({"role": "assistant", "content": ""}))
        return "".join(prompts)


    @staticmethod
    def format_python_prompt_output(system_prompt, user_prompt_1, assistant_response, user_prompt_2) -> str:
        template = f"""from typing import (
    List,
    Literal,
    Sequence,
    TypedDict,
)

Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str


Dialog = Sequence[Message]

class ChatFormat:

    @staticmethod
    def encode_header( message: Message) -> List[str]:
        prompts = []
        prompts.append("<|start_header_id|>")
        prompts.extend(message["role"])
        prompts.append("<|end_header_id|>")
        prompts.extend("\\n\\n")
        return prompts

    @staticmethod
    def encode_message(message: Message) -> List[str]:
        prompts = ChatFormat.encode_header(message)
        prompts.extend(message["content"].strip())
        prompts.append("<|eot_id|>")
        return prompts

    @staticmethod
    def encode_dialog_prompt(dialog: Dialog) -> str:
        prompts = ["<|begin_of_text|>"]
        for message in dialog:
            prompts.extend(ChatFormat.encode_message(message))
        # Add the start of an assistant message for the model to complete.
        prompts.extend(ChatFormat.encode_header({{"role": "assistant", "content": ""}}))
        return "".join(prompts)

if __name__ == "__main__":
    dialog: Dialog = []"""
        if system_prompt:
            system_section = f"""
    dialog.append({{   "role": "system", "content": "{system_prompt}", }})"""
            template += system_section
        user_section = f"""
    dialog.append({{   "role": "user", "content": "{user_prompt_1}", }})"""
        template += user_section
        if assistant_response:
            assistant_section = f"""
    dialog.append({{   "role": "assistant", "content": "{assistant_response}", }})"""
            template += assistant_section
        if user_prompt_2:
            second_user_prompt_section = f"""
    dialog.append({{   "role": "user", "content": "{user_prompt_2}", }})"""
            template += second_user_prompt_section

        template += f"""
    print(ChatFormat.encode_dialog_prompt(dialog))"""

        return template


In [7]:
import gradio as gr

SINGLE_TURN = "Single Turn"
MULTI_TURN = "Multi Turn"

def prompt_template_dropdown_listener(value):
    # This function will be called whenever the value of the dropdown changes
    if value == SINGLE_TURN:
        return {
            assistant_response: gr.Textbox(elem_id="assistant_response", visible = False),
            user_prompt_2: gr.Textbox(elem_id="user_prompt_2", visible = False)
        }
    else:
        return {
            assistant_response: gr.Textbox(elem_id="assistant_response", visible = True),
            user_prompt_2: gr.Textbox(elem_id="user_prompt_2", visible = True)
        }

def format_prompt_template_listener(system_prompt, user_prompt_1, assistant_response, user_prompt_2, prompt_template):
    if not user_prompt_1:
        raise gr.Error("User prompt is mandatory.")

    if not user_prompt_2 and assistant_response:
        raise gr.Error("When the assistant message is set, the second user prompt is mandatory.")

    if prompt_template == MULTI_TURN and ((user_prompt_2 and not assistant_response) or (not user_prompt_2 and not assistant_response)):
        raise gr.Error("When generating a multi turn prompt, the assistant message is mandatory.")

    dialog: Dialog = []
    if system_prompt: dialog.append({   "role": "system", "content": system_prompt, })
    dialog.append({   "role": "user", "content": user_prompt_1, })
    if assistant_response: dialog.append({   "role": "assistant", "content": assistant_response, })
    if user_prompt_2: dialog.append({   "role": "user", "content": user_prompt_2, })

    return {
        prompt_output: ChatFormat.encode_dialog_prompt(dialog),
        python_output: ChatFormat.format_python_prompt_output(system_prompt, user_prompt_1, assistant_response, user_prompt_2),
    }

css = """
h1 {
    text-align: center;
    display:block;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("#  Llama 3 Prompt Formatter")
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            gr.Markdown("## Configurations")
            prompt_template = gr.Dropdown([SINGLE_TURN, MULTI_TURN], label="Prompt Template", filterable=False, value=SINGLE_TURN)

            gr.Markdown("## Input Prompts")
            system_prompt = gr.Textbox(label="System prompt", lines=2, placeholder="Optional System prompt for the model")
            user_prompt_1 = gr.Textbox(elem_id="user_prompt_1", label="User prompt *", lines=2, placeholder="User prompt required")
            assistant_response = gr.Textbox(label="Assistant response", lines=2, visible=False, elem_id="assistant_response", placeholder="Assistant prompt required on Multi Turn")
            user_prompt_2 = gr.Textbox(label="User prompt *", lines=2, visible=False, elem_id="user_prompt_2", placeholder="User prompt required on Multi turn")

            prompt_template.input(prompt_template_dropdown_listener, prompt_template, [assistant_response, user_prompt_2])

            submit = gr.Button("Submit")

        with gr.Column(scale=3, min_width=600):
            gr.Markdown("## Output")
            with gr.Tab("Preview"):
                prompt_output = gr.Code(show_label=True, interactive=False, min_width=600, lines=30)

            with gr.Tab("Code"):
                with gr.Row():
                    with gr.Tab("Python"):
                        python_output = gr.Code(label="Python Code", interactive=False, min_width=600, lines=30, language="python")
                    

    inputs = [system_prompt, user_prompt_1, assistant_response, user_prompt_2, prompt_template]
    outputs = [prompt_output, python_output]
    submit.click(format_prompt_template_listener, inputs, outputs)

demo.queue().launch(server_name="0.0.0.0")


ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7862): address already in use


Running on local URL:  http://0.0.0.0:7865

To create a public link, set `share=True` in `launch()`.


In [8]:
demo.close()

Closing server running on port: 7865
